In [ ]:
#  gene-specific technical noise variance of gene expression
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
from localreg import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol, allen_mouse_genes = LoadGeneINFO()

# 

In [ ]:
JW_ExpMat = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/ExpressionMats/cluster_V3_ExpMat.csv", index_col=0)

In [ ]:
JW_ExpMat.head(2)

In [ ]:
All_Genes_ExpMean = []; All_Genes_ExpVar = []
ALL_Genes_Corr = []
for g, row in JW_ExpMat.iterrows():
    exp_values = np.exp2(row.values)
    mean = np.nanmean(exp_values)
    var = np.nanvar(exp_values)
    All_Genes_ExpMean.append(mean)
    All_Genes_ExpVar.append(var)
    if g in g_corr_df.index.values:
        corr = g_corr_df.loc[g, "Corr"]
    else:
        corr = -1
    ALL_Genes_Corr.append(corr)
All_Genes_ExpMean = np.array(All_Genes_ExpMean)
All_Genes_ExpVar = np.array(All_Genes_ExpVar)
ALL_Genes_Corr = np.array(ALL_Genes_Corr)

In [ ]:
df = pd.DataFrame(data={"Genes":JW_ExpMat.index.values, "ExpMean":All_Genes_ExpMean,
                       "ExpVar":All_Genes_ExpVar, "STRCorr":ALL_Genes_Corr})
df = df.set_index("Genes")

In [ ]:
df2 = df[df["STRCorr"]!=-1]

In [ ]:
X = np.log10(df["ExpMean"].values)
Y = np.log10(df["ExpVar"].values+1)
df3 = df2[df2["STRCorr"]>0.3]
X3 = np.log10(df3["ExpMean"].values)
Y3 = np.log10(df3["ExpVar"].values+1)

df4 = df2[df2["STRCorr"]<0.3]
X4 = np.log10(df4["ExpMean"].values)
Y4 = np.log10(df4["ExpVar"].values+1)

In [ ]:
fig, axs = plt.subplots(1,2,dpi=120, figsize=(12,5))
axs[0].scatter(X, Y, s=0.1, alpha=0.5, color="grey")
axs[0].scatter(X3, Y3, s=0.5, alpha=0.5, color="red", label="Corr > 0.3")
axs[0].legend()
axs[0].set_xlabel("log Exp")
axs[0].set_ylabel("log Var")

axs[1].scatter(X, Y, s=0.1, alpha=0.5, color="grey")
axs[1].scatter(X4, Y4, s=0.5, alpha=0.5, color="green", label="Corr < 0.3")
axs[1].legend()
axs[1].set_xlabel("log Exp")
axs[1].set_ylabel("log Var")

In [ ]:
X = np.log2(All_Genes_ExpMean+1)
Y = np.log2(All_Genes_ExpVar+1)

In [ ]:
X_srt_idx = np.argsort(X)
X_sorted = X[X_srt_idx]
Y_sorted = Y[X_srt_idx]

In [ ]:
plt.figure(dpi=120)
plt.scatter(X_sorted, Y_sorted, s=0.1, alpha=0.5)

Y2 = localreg(X_sorted, Y_sorted, degree=2, kernel=rbf.tricube, radius=0.3)
plt.plot(X_sorted, Y2, label='Local quadratic regression', color="red")
plt.legend()

In [ ]:
Y2_orginal = np.exp2(Y2)-1
Y2_orginal[Y2_orginal<0]=0

In [ ]:
min(Y2_orginal)

In [ ]:
def Cal_Tech_Frac(VarTech, VarTotal):
    res = []
    for v1, v2 in zip(VarTech, VarTotal):
        if v2 == 0:
            res.append(1)
        else:
            res.append(v1/v2)
    return np.array(res)
TechVarFrac = Cal_Tech_Frac(Y2_orginal, All_Genes_ExpVar[X_srt_idx])

In [ ]:
TechVarFrac[TechVarFrac>1]=1

In [ ]:
TechVarFrac

In [ ]:
print(min(TechVarFrac), max(TechVarFrac))

In [ ]:
plt.scatter(X_sorted, TechVarFrac, s=0.1, alpha=0.5)

In [ ]:
TechVar = Y * TechVarFrac

In [ ]:
genes = JW_ExpMat.index.values
genes_sorted = genes[X_srt_idx]
print(len(genes_sorted), len(TechVar))

In [ ]:
gene_techVar_dict = dict(zip(genes_sorted, TechVarFrac))

In [ ]:
# next step: check corr ISH/SC has alterd var frac?

In [ ]:
g_corr_df = pd.read_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.csv", index_col="Genes")

In [ ]:
g_corr_df.head(2)

In [ ]:
for i, row in g_corr_df.iterrows():
    g_corr_df.loc[i, "techVarFrac"] = gene_techVar_dict[i]

In [ ]:
plt.figure(dpi=120)
plt.scatter(g_corr_df["Corr"].values, g_corr_df["techVarFrac"].values, s=0.1)
plt.xlabel("STR Corr")
plt.ylabel("Frac Tech Var")

In [ ]:
g_corr_df

# Use V2 V3 Diff to denoise

In [ ]:
V2_ExpMat = pd.read_csv("dat/ExpressionMats/Subclass_V2_ExpMat.csv", index_col=0)
V3_ExpMat = pd.read_csv("dat/ExpressionMats/Subclass_V3_ExpMat.csv", index_col=0)

In [ ]:
V2_ExpMat.head(2)

In [ ]:
V3_ExpMat.head(2)

In [ ]:
Corss_CT_Corrs = []
for g, row in V2_ExpMat.iterrows():
    v2_exps = V2_ExpMat.loc[g, :].values
    v3_exps = V3_ExpMat.loc[g, :].values
    r,p = spearmanr(v2_exps, v3_exps, nan_policy="omit")
    #r,p = spearmanr(v2_exps, v3_exps)
    Corss_CT_Corrs.append(r)

In [ ]:
r,p = spearmanr(v2_exps, v3_exps, nan_policy='omit')

In [ ]:
plt.hist(Corss_CT_Corrs)

In [ ]:
sns.set(style="whitegrid", context="talk")

# Create the figure
plt.figure(dpi=480, figsize=(10, 6))

# Plot the histogram with improved aesthetics
sns.histplot(Corss_CT_Corrs, bins=30, kde=True, color='skyblue', edgecolor='black', alpha=0.7)

# Add labels and title with appropriate font sizes
plt.xlabel("Cross Cell Type Correlations", fontsize=14, weight='bold')
plt.ylabel("Frequency", fontsize=14, weight='bold')
plt.title("Distribution of Cross Cell Type Correlations", fontsize=16, weight='bold')

# Add grid lines for better readability
plt.grid(True, linestyle='--', alpha=0.5)

# Improve tick parameters
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Use tight layout to avoid clipping
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
ISH_SC_CorrDF = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/ISH_MERFISH_Gene_CorssSTR_Corr.csv", index_col=0)

In [ ]:
ISH_SC_CorrDF.head(2)

In [ ]:
for g, row in ISH_SC_CorrDF.iterrows():
    v2_exps = V2_ExpMat.loc[g, :].values
    v3_exps = V3_ExpMat.loc[g, :].values
    r,p = spearmanr(v2_exps, v3_exps, nan_policy="omit")
    ISH_SC_CorrDF.loc[g, "V2_V3_CT_Corr"] = r

In [ ]:
ISH_SC_CorrDF.head(2)

In [ ]:
plt.scatter(ISH_SC_CorrDF["Corr"].values, ISH_SC_CorrDF["V2_V3_CT_Corr"].values, s=0.1)
plt.xlabel("ISH-SC STR Corr")
plt.ylabel("V2 V3 CT Corr")

In [ ]:
pearsonr(ISH_SC_CorrDF["Corr"].values, ISH_SC_CorrDF["V2_V3_CT_Corr"].values)

In [ ]:
ISH_SC_CorrDF.to_csv("dat/ISH_MERFISH_Gene_CorssSTR_Corr.v2.csv")

In [ ]:
15000 / 200

In [ ]:
15000 * 10

In [ ]:
150000 / 12800 * 32

In [ ]:
15000 / 160

In [ ]:
93 * (0.75395 * 1 + 0.20105 * 4 + 0.045 * 32)

In [ ]:
93 * (0.75395 * 600 + 0.20105 * 3200 + 0.045 * 35000) * 0.85

In [ ]:
211121.2065 / 12800 * 32

In [ ]:
(91 * 600 + 2 * 35000) * 0.85

In [ ]:
105910.0 / 12800 * 32